- autor: Eduardo Pacheco (edu@atlasia.tech)
- 11/01/2023

### Estudo de Caso Dotz

Precisamos entenderrt o algoritmo SHAP e a fórmula do SHAP values no paper original. A abordagem que daremos aqui é uma abordagem computacional, de forma a reproduzir os exemplos do artigo  [SHAP Values Explained Exactly How You Wished Someone Explained to You](https://towardsdatascience.com/shap-explained-the-way-i-wish-someone-explained-it-to-me-ab81cc69ef30) via código.


### O que é SHAP (resumo cego)?

O algoritmo SHAP (SHapley Additive exPlanations ) é um modelo de explicabilidade de modelos de machine learning aplicável a modelos black box. Ele é capaz de medir a contribuição marginal de cada feature em um caso na previsão daquele caso. É, portanto, um modelo de explicabilidade local.

SHAP foi proposto por Lundberg e Lee no artigo [A Unified Approach to Interpreting Model Predictions](https://arxiv.org/abs/1705.07874). Os autores se valem de um conceito das teorias dos jogos chamado Sharpley Values. Os Sharpley Values são valores associados à importância de cada jogador em um jogo. Essa importância é calculada considerando cada possível interação entre jogadores e os resultados a partir dessa combinação. Em outras palavras : para determinar a importãncia de um jogador no jogo deve-se considerar cada possível combinação de jogadores  (ou coalizão ).

O que o SHAP faz é uma metáfora dessa ideia considerando que o jogo é acertar as previsões de um modelo de machine learning ( usualmente black box) e os jogadores são as features. A partir disso, considera-se todas as possíveis combinações entre features (em um power set de features) e se produz um modelo para cada combinação, incluíndo aí o modelo nulo ou dummy (sem features). Dessa forma, é possível saber a previsão feita por cada um desses modelos que partem do mesmo caso (X0) e que usam os mesmos hiperparâmetros - mas que diferem entre si no conjunto de features usado.

De posse disso, é possível criar contrastes entre dois dados modelos(quando aplicados a um caso X0, frise-se) que são treinados em condições idênticas exceto pela presença de uma feature. Essa diferença na presença dessa feature leva, naturalmente, a uma diferença no funcionamento interno do modelo e na previsão que é realizada por ele nesse caso X0. Para calcular o SHAP, a diferença entre as previsões é atribuída ("imputada") à presença (e à extensão) daquela feature. Assim, o tamanho da diferença entre as previsões dos modelos com e sem a feature considerada é a **contribuição marginal** dessa feature nesse caso (X0) **para esse modelo** (que possui essa feature, não presente no modelo-contraste). 

O próximo passo é encontrar essa contribuição marginal para todos os modelos que possuem essa feature adicionada e repetir o cálculo da contribuição marginal dessa feature em cada um desses modelos. Com esses valores podemos criar uma equação que equipara ao valor um todas as contribuições marginais dessa feature em cada um dos modelos, atribuídas um peso para cada uma e somadas.

Teremos, aí, tantos pesos  quantos forem os modelos em que aquela feature está presente e igual número de contribuições marginais. As contribuições marginais da feature no modelo com ela já estarão a essa altura calculadas e os pesos são desconhecidos. Para descobrir os pesos pela via heurística (computacional) é possível aplicar um algoritmo como o a seguir:

1. Considere F o número de features do modelo a ser explicado.
2. Defina uma lista de 0 até F, com cada valor sendo f, representando a quantiadde de features que aquele modelo usa ao ser treinado.
3. Crie um powerset de features, a partir do conjunto de features usado no modelo original a ser previsto, de modo que todas as combinações de features, incluindo o conjunto vazio, apareça ( e que cada feature apareça apenas uma vez).
4. Treine um modelo para cada combinação possível de features e os aloque/associe ao valor f, como se cada modelo estivesse associado a um "nível" de f.
5. Crie um peso para cada modelo treinado (w1, w2, w3).
6. Iguale a somatória dos pesos a 1.
7. Iguale a soma dos pesos de todas as camadas (f=0, f=1, f=2...) entre si.
8. Iguale os pesos dentro de cada camada.
9. Encontre a solução considerando 6, 7 e 8. Isso dará as os valores dos pesos associados a cada contribuição marginal daquela feature em cada modelo em que ela está presente. 

Agora temos tanto os pesos quanto as contribuições marginais. Aplicando os pesos nas contribuiçoes marginais e somando tudo teremos o valor SHAP associado a essa feature para esse caso X0. Ao repetir o processo para todas as features teremos a importância de cada uma delas na previsão ( ou melhor, na explicação da diferença entre o valor com as featurese o modelo nulo).

### Reprodução via código do SHAP

Vamos reproduzir via código a ideia geral do SHAP ( sem incluir as otimizações, para fins meramente didáticos). Consideraremos os valores do site para o exemplo de um modelo que prevê o salário dado age, gender e job. Fizemos um arquivo CSV para representar cada modelo sendo que cada feature é um binário, exceto pela feature prevista, que é o valor do salário.   

In [131]:
import pandas as pd

# carrega um dataframe com as representações dos modelos d0 powerset de features e suas previsões
models = pd.read_csv('models_preds.csv')

#pega as features, sua quantidade e a cardinalidade
features = list(models.columns)
X,y = features[0:-1], features[-1]
n_features = len(X)
cardinality = 2**n_features

models

,age,gender,job,salary
0,0,0,0,50
1,1,0,0,40
2,0,1,0,48
3,0,0,1,100
4,1,1,0,39
5,1,0,1,85
6,0,1,1,95
7,1,1,1,83


O próximo passo é conseguir descobrir o valor de f para cada modelo representado. Em seguida vamos descobrir o edge que vincula um modelo a outro, de forma a produzir uma árvore.

In [132]:
def get_f(model):
    ''' recebe a representação de um modelo (Series) e retorna o valor de f para ele'''
    return  int(model[0:-1].sum())

In [133]:
m0 = models.query('age == 0 and gender == 0 and job == 0').squeeze()
f = get_f(m0)
print('\nO modelo \n',m0, '\ntem f = ', f)

m4 = models.query('age == 1 and gender == 1 and job == 0').squeeze()
f = get_f(m4)
print('\nO modelo \n',m4, '\ntem f = ', f)

m7 = models.query('age == 1 and gender == 1 and job == 1').squeeze()
f = get_f(m7)
print('\nO modelo \n',m7, '\ntem f = ', f)


O modelo 
 age        0
gender     0
job        0
salary    50
Name: 0, dtype: int64 
tem f =  0

O modelo 
 age        1
gender     1
job        0
salary    39
Name: 4, dtype: int64 
tem f =  2

O modelo 
 age        1
gender     1
job        1
salary    83
Name: 7, dtype: int64 
tem f =  3


A função para achar o f funciona. Agora vamos criar um grafo unidirecional/árvore que represente os edges. Para isso precisamos catalogar os modelos, atribuir os valores de f, olhar para todos os f +1 e ver se o modelo ali possui todas as suas features e, se sim, se a feature adicional é a feature alvo. Se for, então há um edge entre o modelo no nível f e o nível f+1. Esse edge é importante porque é a partir desse vínculo que tanto pegamos a contribuição marginal da feature naquele modelo para o caso X0 quanto sabemos que precisamos ter um peso associado a essa contribuição marginal.

In [152]:
def get_fs(models):
    ''' função registrar o valor de f'''
    models_catag = models.copy()
    #registra o f
    for ix, row in models_catag.iterrows():
        models_catag.at[ix,'f'] = get_f(row)
    return models_catag

def get_edges(feature_alvo,X, m):
    ''' função para criar a árvore de conexões entre representações de modelo, dada uma feature alvo e um conjunto de features'''
    models_catag = m.copy()
    f = ['f']
    features_nao_alvo = [feature for feature in X if feature not in feature_alvo and feature not in f]
    print(features_nao_alvo)
    # varre cada modelo e descobre suas conexões
    for ix, row in models_catag.iterrows():
        #a origem não pode ter a feature alvo, se tiver pode passar
        if not row[feature_alvo]:
            f_from = row.f 
            #pega o caso que é idêntico nas features não alvo, com f+1
            query = 'f == {} '.format(f_from + 1)
            #monta uma string para a query que encontra o próximo nó
            for feature in features_nao_alvo:
                query = query +' and '+ feature + ' == {} '.format(row[feature])
            # pega o próximo nó para montar o edge
            ix_edge = models_catag.query(query).index[0]
            models_catag.at[ix,'edge'] = ix_edge
    return models_catag

Se tudo correr bem então os resultados são de tal forma que, dado uma feature alvo, um modelo aponte para outro modelo idêntico nas demais features, mas que o modelo destino tenha a feature alvo e o origem não. Vamos testar para os 3 casos.

In [153]:
models_f = get_fs(models)
edges_gender = get_edges('gender',X, models_f)
edges_gender

['age', 'job']


,age,gender,job,salary,f,edge
0,0,0,0,50,0.0,2.0
1,1,0,0,40,1.0,4.0
2,0,1,0,48,1.0,NaN
3,0,0,1,100,1.0,6.0
4,1,1,0,39,2.0,NaN
5,1,0,1,85,2.0,7.0
6,0,1,1,95,2.0,NaN
7,1,1,1,83,3.0,NaN


In [136]:
edges_age = get_edges('age',X, models_f)
edges_age

['gender', 'job']


,age,gender,job,salary,f,edge
0,0,0,0,50,0.0,1.0
1,1,0,0,40,1.0,NaN
2,0,1,0,48,1.0,4.0
3,0,0,1,100,1.0,5.0
4,1,1,0,39,2.0,NaN
5,1,0,1,85,2.0,NaN
6,0,1,1,95,2.0,7.0
7,1,1,1,83,3.0,NaN


In [137]:
edges_job = get_edges('job',X, models_f)
edges_job

['age', 'gender']


,age,gender,job,salary,f,edge
0,0,0,0,50,0.0,3.0
1,1,0,0,40,1.0,5.0
2,0,1,0,48,1.0,6.0
3,0,0,1,100,1.0,NaN
4,1,1,0,39,2.0,7.0
5,1,0,1,85,2.0,NaN
6,0,1,1,95,2.0,NaN
7,1,1,1,83,3.0,NaN


Tudo certo aqui. Agora vamos calcular as contribuições marginais de uma feature em cada modelo.

In [138]:
import math

def get_MC(feature_alvo,X, edges):
    ''' retorna as contribuições marginais associadas a cada conexão entre modelos do contraste com e sem a feature alvo '''
    marginal_contributions = edges.copy()

    for ix, row in marginal_contributions.iterrows():
        #verifica se o modelo atual se conecta a outro
        if not math.isnan(row.edge):
            #se sim pega o salário de origem e destino
            prev_to = marginal_contributions.iloc[int(row.edge)].salary
            prev_from = row.salary

            #calcula a diferença e atribui como contribuição marginal ao modelo com a feature alvo
            diff = prev_to - prev_from
            marginal_contributions.at[int(row.edge),'MC'] = diff
    return marginal_contributions

In [139]:
age_MC = get_MC('age',X, edges_age)
age_MC

,age,gender,job,salary,f,edge,MC
0,0,0,0,50,0.0,1.0,NaN
1,1,0,0,40,1.0,NaN,-10.0
2,0,1,0,48,1.0,4.0,NaN
3,0,0,1,100,1.0,5.0,NaN
4,1,1,0,39,2.0,NaN,-9.0
5,1,0,1,85,2.0,NaN,-15.0
6,0,1,1,95,2.0,7.0,NaN
7,1,1,1,83,3.0,NaN,-12.0


Para age bate com os exemplos do [artigo de Samuelle Mazantinni : SHAP Values Explained Exactly How You Wished Someone Explained to You] (https://towardsdatascience.com/shap-explained-the-way-i-wish-someone-explained-it-to-me-ab81cc69ef30). Vejamos para as outras features.

In [140]:
job_MC = get_MC('job',X, edges_job)
job_MC

,age,gender,job,salary,f,edge,MC
0,0,0,0,50,0.0,3.0,NaN
1,1,0,0,40,1.0,5.0,NaN
2,0,1,0,48,1.0,6.0,NaN
3,0,0,1,100,1.0,NaN,50.0
4,1,1,0,39,2.0,7.0,NaN
5,1,0,1,85,2.0,NaN,45.0
6,0,1,1,95,2.0,NaN,47.0
7,1,1,1,83,3.0,NaN,44.0


In [141]:
gender_MC = get_MC('gender',X, edges_gender)
gender_MC

,age,gender,job,salary,f,edge,MC
0,0,0,0,50,0.0,2.0,NaN
1,1,0,0,40,1.0,4.0,NaN
2,0,1,0,48,1.0,NaN,-2.0
3,0,0,1,100,1.0,6.0,NaN
4,1,1,0,39,2.0,NaN,-1.0
5,1,0,1,85,2.0,7.0,NaN
6,0,1,1,95,2.0,NaN,-5.0
7,1,1,1,83,3.0,NaN,-2.0


### Calculando os pesos

Para calcular os pesos de modo heurístico (e não pela via das fórmulas ou com binomial) precisamos respeitar trẽs regras:

1. A soma de todos os pesos é igual a 1;
2. A soma de cada camada (f) de features obtidas no poweset é igual a cada outra camada;
3. O valor dos pesos **dentro** de cada camada é o mesmo para cada modelo nessa camada.

In [142]:
def get_weights(edges):
    ''' devolve os pesos dado um DF de MC '''
    # filtra apenas os casos que já tenham MC
    weights = edges.query('not MC.isnull()', engine='python')
    #pega todos os fs
    fs = weights.f.unique()
    n_fs = len(fs)

    #varre os modelos com MC e conta quantos modelos tem naquele f
    for ix, row in weights.iterrows():
        n_models = len(weights.query('f == {}'.format(row.f)))
        # atribui o peso w
        weights.at[ix,'w'] = 1 / (n_fs * n_models)

    return weights


In [143]:
# pega os pesos
age_mc_w = get_weights(age_MC)
gender_mc_w = get_weights(gender_MC)
job_mc_w = get_weights(job_MC)

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Calculamos os pesos, agora vamos somar tudo.

In [144]:
import numpy as np

def get_feature_shap(mc_w):
    ''' faz a somatória dado um df com MC e w '''
    return np.round(np.sum(np.multiply(np.array(mc_w['MC']), np.array(mc_w['w']))),2)


In [145]:
age_shap = get_feature_shap(age_mc_w)
age_shap

-11.33

In [146]:
gender_shap = get_feature_shap(gender_mc_w)
gender_shap

-2.33

In [147]:
job_shap = get_feature_shap(job_mc_w)
job_shap

46.67

Os valores bateram com os do artigo didático. Agora vamos devolver um pandas dataframe com os SHAP values.

In [162]:
def get_SHAP_df(model):
    ''' dado um dataframe com a representação dos modelos retorna um df com os SHAP values'''

    #vamos assumir que apenas a representação dos modelos foi passada então pegaremos as features novamente
    #pega as features, sua quantidade e a cardinalidade
    features = list(model.columns)
    X,y = features[0:-1], features[-1]
    n_features = len(X)
    cardinality = 2**n_features
    #pega os valores de f
    model_f = get_fs(model)
    # filtra o modelo que usa todas as features, que tem f máximo
    shap_df = model_f.query('f == {}'.format(n_features))

    #converte as features para float
    for x in X:
        shap_df[x] = shap_df[x].astype(float)
    index = shap_df.head(1).index[0]
    #para cada feature
    for feature in X:
        #pega as conexões
        edges = get_edges(feature,X, model_f)
        #calcula as contribuições marginais
        mc = get_MC(feature,X, edges)
        #agrega os pesos
        mc_w = get_weights(mc)
        #calcula a feature SHAP
        feature_shap = get_feature_shap(mc_w)
        
        shap_df.at[index,feature] = feature_shap
    return shap_df.drop('f', axis=1)



In [163]:
models = pd.read_csv('models_preds.csv')
get_SHAP_df(models)

<ipython-input-162-a85cb63453f3>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shap_df[x] = shap_df[x].astype(float)
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/anaconda/envs/azureml_py38/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

,age,gender,job,salary
7,-11.33,-2.33,46.67,83


### Conclusão

Conseguimos reproduzir computacionalmente a fórmula de cálculo dos valores SHAP. Isso implica em uma compreensão de sua mecânica e usa a mesma lógica. O código aqui usou como exemplos o artigo [SHAP Values Explained Exactly How You Wished Someone Explained to You](https://towardsdatascience.com/shap-explained-the-way-i-wish-someone-explained-it-to-me-ab81cc69ef30) de Samuele Mazzanti e usa a abordagem da série Think (como em [Think Statistics](https://greenteapress.com/wp/think-stats-2e/), que resolve problemas de estatística com uma abordagem computacional) e do livro Data Science do Zero.

É possível a partir daqui adaptar esse código para pegar e treinar modelos reais.